In [2]:
# start coding here

In [3]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

import altair as alt
from altair_saver import save as alt_save

## Does the first principal component just align to the axis of overall metastatic potential?

In [4]:
metmap_tissues = [
    'brain',
    'lung',
    'liver',
    'bone',
    'kidney'
]

In [5]:
all_df = pd.read_excel(snakemake.input['mm_potential'], sheet_name=f"metp500.all5", index_col=0)
all_df.head()

In [6]:
met_potential_df = pd.DataFrame(index=all_df.index.values.tolist())

In [7]:
for metmap_tissue in metmap_tissues:
    tissue_df = pd.read_excel(snakemake.input['mm_potential'], sheet_name=f"metp500.{metmap_tissue}", index_col=0)
    met_potential_df[metmap_tissue] = tissue_df['mean']

In [8]:
met_potential_df.head()

In [9]:
X = met_potential_df.values
pca = PCA(n_components=2)
X_prime = pca.fit_transform(X)

In [10]:
pca.explained_variance_ratio_

In [11]:
pca.singular_values_

In [12]:
pca_df = pd.DataFrame(index=all_df.index.values.tolist(), data=X_prime, columns=["PC1", "PC2"])
pca_df["all5_mean"] = all_df["mean"]
pca_df["all5_penetrance"] = all_df["penetrance"]
pca_df["all5_penetrance_is_zero"] = (all_df["penetrance"] == 0.0)

pca_df = pca_df.reset_index()

In [13]:
pca_df.head()

In [14]:
(all_df["penetrance"] != 0.0).sum()

In [15]:
scales = alt.selection_interval(bind='scales')

penetrance_plot = alt.Chart(pca_df).mark_circle(size=40, opacity=1).encode(
    x=alt.X("PC2:Q"),
    y=alt.Y("PC1:Q"),
    color=alt.Color("all5_penetrance_is_zero:N", legend=alt.Legend(title="Metastasis Penetrance is Zero"), scale=alt.Scale(domain=[False, True])),
    tooltip=[
        alt.Tooltip("index:N", title="Cell Line"),
        alt.Tooltip("all5_mean:Q", title="Overall Metastasis Potential"),
        alt.Tooltip("all5_penetrance:Q", title="Overall Metastasis Penetrance")
    ]
).add_selection(
    scales
).properties(
    width=500,
    height=500
)
penetrance_plot

In [16]:
alt_save(penetrance_plot, snakemake.output['pca_plot'])

In [17]:
scales = alt.selection_interval(bind='scales')

alt.Chart(pca_df).mark_circle(size=40, opacity=1).encode(
    x=alt.X("PC2:Q"),
    y=alt.Y("PC1:Q"),
    color=alt.Color("all5_mean:Q", legend=alt.Legend(title="Overall Metastasis Potential")),
    tooltip=[
        alt.Tooltip("index:N", title="Cell Line"),
        alt.Tooltip("all5_mean:Q", title="Overall Metastasis Potential")
    ]
).add_selection(
    scales
).properties(
    width=500,
    height=500
)

In [18]:
scales = alt.selection_interval(bind='scales')

alt.Chart(pca_df).mark_circle(size=40, opacity=1).encode(
    x=alt.X("PC2:Q"),
    y=alt.Y("PC1:Q"),
    color=alt.Color("all5_mean:Q", legend=alt.Legend(title="Overall Metastasis Potential"), scale=alt.Scale(domain=[-4, -3])),
    tooltip=[
        alt.Tooltip("index:N", title="Cell Line"),
        alt.Tooltip("all5_mean:Q", title="Overall Metastasis Potential")
    ]
).add_selection(
    scales
).properties(
    width=500,
    height=500
)

# Filtering out cell lines with `penetrance == 0`

372 cell lines have non-zero overall penetrance

In [19]:
nonzero_potential_df = met_potential_df.loc[all_df['penetrance'] != 0]
nonzero_potential_df.head()

In [20]:
nonzero_all_df = all_df.loc[all_df['penetrance'] != 0]

In [21]:
X = nonzero_potential_df.values
pca = PCA(n_components=2)
X_prime = pca.fit_transform(X)

In [22]:
pca.explained_variance_ratio_

In [59]:
nonzero_pca_df = pd.DataFrame(index=nonzero_potential_df.index.values.tolist(), data=X_prime, columns=["PC1", "PC2"])
nonzero_pca_df["all5_mean"] = nonzero_all_df["mean"]
nonzero_pca_df["all5_penetrance"] = nonzero_all_df["penetrance"]

nonzero_pca_df = nonzero_pca_df.merge(nonzero_potential_df, how="left", left_index=True, right_index=True)
nonzero_pca_df = nonzero_pca_df.reset_index()

In [60]:
nonzero_potential_plot = alt.Chart(nonzero_pca_df).mark_circle(size=60, opacity=1).encode(
    x=alt.X("PC2:Q"),
    y=alt.Y("PC1:Q"),
    color=alt.Color("all5_mean:Q", legend=alt.Legend(title="Overall Metastasis Potential")),
    tooltip=[
        alt.Tooltip("index:N", title="Cell Line"),
        alt.Tooltip("all5_mean:Q", title="Overall Metastasis Potential")
    ]
).properties(
    title="Principal Components: Overall Metastasis Potential",
    width=500,
    height=500
)

nonzero_penetrance_plot = alt.Chart(nonzero_pca_df).mark_circle(size=60, opacity=1).encode(
    x=alt.X("PC2:Q"),
    y=alt.Y("PC1:Q"),
    color=alt.Color("all5_penetrance:Q", legend=alt.Legend(title="Overall Metastasis Penetrance")),
    tooltip=[
        alt.Tooltip("index:N", title="Cell Line"),
        alt.Tooltip("all5_mean:Q", title="Overall Metastasis Potential")
    ]
).properties(
    title="Principal Components: Overall Metastasis Potential",
    width=500,
    height=500
)

alt.hconcat(nonzero_potential_plot, nonzero_penetrance_plot).resolve_scale(
    color='independent'
)

In [76]:
tooltip = [
    alt.Tooltip("index:N", title="Cell Line"),
    alt.Tooltip("all5_mean:Q", title="Potential"),
    alt.Tooltip("all5_penetrance:Q", title="Penetrance")
]

nonzero_potential_pc1_plot = alt.Chart(nonzero_pca_df).mark_circle(size=60, opacity=1).encode(
    x=alt.X("all5_mean:Q"),
    y=alt.Y("PC1:Q"),
    color=alt.Color("all5_mean:Q", legend=alt.Legend(title="Potential")),
    tooltip=tooltip
).properties(
    title="PC1 vs. Potential",
    width=400,
    height=400
)

nonzero_penetrance_pc1_plot = alt.Chart(nonzero_pca_df).mark_circle(size=60, opacity=1).encode(
    x=alt.X("all5_penetrance:Q"),
    y=alt.Y("PC1:Q"),
    color=alt.Color("all5_penetrance:Q", legend=alt.Legend(title="Penetrance")),
    tooltip=tooltip
).properties(
    title="PC1 vs. Penetrance",
    width=400,
    height=400
)

nonzero_potential_pc2_plot = alt.Chart(nonzero_pca_df).mark_circle(size=60, opacity=1).encode(
    x=alt.X("all5_mean:Q"),
    y=alt.Y("PC2:Q"),
    color=alt.Color("all5_mean:Q", legend=alt.Legend(title="Potential")),
    tooltip=tooltip
).properties(
    title="PC2 vs. Potential",
    width=400,
    height=400
)

nonzero_penetrance_pc2_plot = alt.Chart(nonzero_pca_df).mark_circle(size=60, opacity=1).encode(
    x=alt.X("all5_penetrance:Q"),
    y=alt.Y("PC2:Q"),
    color=alt.Color("all5_penetrance:Q", legend=alt.Legend(title="Penetrance")),
    tooltip=tooltip
).properties(
    title="PC2 vs. Penetrance",
    width=400,
    height=400
)

alt.vconcat(alt.hconcat(nonzero_potential_pc1_plot, nonzero_penetrance_pc1_plot).resolve_scale(
    color='independent'
), alt.hconcat(nonzero_potential_pc2_plot, nonzero_penetrance_pc2_plot).resolve_scale(
    color='independent'
)).resolve_scale(
    color='independent'
)

In [75]:
def get_tissue_plot(tissue):
    return alt.Chart(nonzero_pca_df).mark_circle(size=40, opacity=1).encode(
        x=alt.X(f"{tissue}:Q"),
        y=alt.Y("PC1:Q"),
        color=alt.Color("all5_mean:Q", legend=alt.Legend(title="Overall Potential"), scale=alt.Scale(domain=[-4, 3])),
        tooltip=tooltip
    ).properties(
        title=f"PC1 vs. {tissue} potential",
        width=300,
        height=300
    )

brain_plot = get_tissue_plot("brain")
lung_plot = get_tissue_plot("lung")
liver_plot = get_tissue_plot("liver")
bone_plot = get_tissue_plot("bone")
kidney_plot = get_tissue_plot("kidney")

alt.hconcat(brain_plot, lung_plot, liver_plot, bone_plot, kidney_plot)